# emotional coloring

In [3]:
#pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#pip install protobuf

In [21]:
import tensorflow as tf

In [1]:
import numpy as np
import re

from tensorflow.keras.layers import Dense, LSTM, Input, Dropout, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

#стемминг
class Porter:
    PERFECTIVEGROUND = re.compile(u"((лык|луу|ла|ин|ун|дуу)|((?<=[ая])(в|вши|вшись)))$")
    REFLEXIVE = re.compile(u"(с[яь])$")
    ADJECTIVE = re.compile(u"(да|до|де|дө|дан|дон|ден|дөн|ны|ну|ни|нү|нын|нун|нин|нүн)$")
    PARTICIPLE = re.compile(u"((лар|лер|лор|лөр)|((?<=[ая])(дар|дер|дор|дөр)))$")
    VERB = re.compile(
        u"((тар|тер|тор|төр|га|ге|го|гө|ка|ке|ко|кө|на||не|ило|ыло|ено|ят|ует|уют|ит|ыт|ены|ить|ыть|ишь|ую|ю)|((?<=[ая])(ла|на|ете|йте|ли|й|л|ем|н|ло|но|ет|ют|ны|ть|ешь|нно)))$")
    NOUN = re.compile(
        u"(а|ев|ов|ие|ье|е|иями|ями|ами|еи|ии|и|ией|ей|ой|ий|й|иям|ям|ием|ем|ам|ом|о|у|ах|иях|ях|ы|ь|ию|ью|ю|ия|ья|я)$")
    RVRE = re.compile(u"^(.*?[аеиоуыэюяө])(.*)$")
    DERIVATIONAL = re.compile(u".*[^аеиоуыэюяө]+[аеиоуыэюяө]$")
    SUPERLATIVE = re.compile(u"(ум|быз)$")
    I = re.compile(u"и$")
    P = re.compile(u"ь$")
    NN = re.compile(u"нн$")

    def stem(word):
        word = word.lower()
        word = word.replace(u'ё', u'е')
        m = re.match(Porter.RVRE, word)
        if m and m.groups():
            pre = m.group(1)
            rv = m.group(2)
            temp = Porter.PERFECTIVEGROUND.sub('', rv, 1)
            if temp == rv:
                rv = Porter.REFLEXIVE.sub('', rv, 1)
                temp = Porter.ADJECTIVE.sub('', rv, 1)
                if temp != rv:
                    rv = temp
                    rv = Porter.PARTICIPLE.sub('', rv, 1)
                else:
                    temp = Porter.VERB.sub('', rv, 1)
                    if temp == rv:
                        rv = Porter.NOUN.sub('', rv, 1)
                    else:
                        rv = temp
            else:
                rv = temp

            rv = Porter.I.sub('', rv, 1)

            if re.match(Porter.DERIVATIONAL, rv):
                rv = Porter.DER.sub('', rv, 1)

            temp = Porter.P.sub('', rv, 1)
            if temp == rv:
                rv = Porter.SUPERLATIVE.sub('', rv, 1)
                rv = Porter.NN.sub(u'н', rv, 1)
            else:
                rv = temp
            word = pre + rv
        return word

    stem = staticmethod(stem)


if __name__ == '__main__':
 #   print(Porter.stem('прибывший'))

    with open(Porter.stem('эмоциялар/оң лексика.txt'), 'r', encoding='utf-8') as f:
        texts_true = f.readlines()
        texts_true[0] = texts_true[0].replace('\ufeff', '') #убираем первый невидимый символ

    with open(Porter.stem('эмоциялар/терс лексика.txt'), 'r', encoding='utf-8') as f:
        texts_false = f.readlines()
        texts_false[0] = texts_false[0].replace('\ufeff', '') #убираем первый невидимый символ

    texts = texts_true + texts_false
    count_true = len(texts_true)
    count_false = len(texts_false)
    total_lines = count_true + count_false
    print(count_true, count_false, total_lines)


    maxWordsCount = 10000
    tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', 
                          lower=True, split=' ', char_level=False)
    tokenizer.fit_on_texts(texts)

    dist = list(tokenizer.word_counts.items())
    print(dist[:10])
    print(texts[0][:100])


    max_text_len = 10
    data = tokenizer.texts_to_sequences(texts)
    data_pad = pad_sequences(data, maxlen=max_text_len)
    print(data_pad)

    print( list(tokenizer.word_index.items()) )


    X = data_pad
    Y = np.array([[1, 0]]*count_true + [[0, 1]]*count_false)
    print(X.shape, Y.shape)

    indeces = np.random.choice(X.shape[0], size=X.shape[0], replace=False)
    X = X[indeces]
    Y = Y[indeces]

    model = Sequential()
    model.add(Embedding(maxWordsCount, 128, input_length = max_text_len))
    model.add(LSTM(128, return_sequences=True))
    model.add(LSTM(64))
    model.add(Dense(2, activation='softmax'))
    model.summary()

    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(0.0001))

    history = model.fit(X, Y, batch_size=5, epochs=50)

    reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

    def sequence_to_text(list_of_indices):
        words = [reverse_word_map.get(letter) for letter in list_of_indices]
        return(words)

    t = "Кутман күн, азыр баарын тактап беребиз, катачылык үчүн кечиресиз.".lower()
    data = tokenizer.texts_to_sequences([t])
    data_pad = pad_sequences(data, maxlen=max_text_len)
    print(sequence_to_text(data[0]))
    
    f = "Биз катаны түшүндүк, азыр баарын тактап беребиз, кайра бул жакка чалбагыла ".lower()
    data2 = tokenizer.texts_to_sequences([f])
    data_pad2 = pad_sequences(data2, maxlen=max_text_len)
    print(sequence_to_text(data2[0]))

    res = model.predict(data_pad)
    print(res, np.argmax(res), sep='\n')
    res2 = model.predict(data_pad2)
    print(res2, np.argmax(res2), sep='\n')

120 73 193
[('аттракциондордун', 1), ('аракети', 1), ('таптакыр', 1), ('ыңгайлашкан', 1), ('тыкан', 3), ('иштетилген', 1), ('активдүү', 3), ('альтруист', 1), ('периштелик', 2), ('чыныгы', 2)]
аттракциондордун аракети таптакыр, ыңгайлашкан, тыкан, тыкан, иштетилген,

[[  0   0   0 ...  39  39 201]
 [  0   0  40 ... 204  40  82]
 [ 39  80  82 ...  40 205  83]
 ...
 [  0   0   0 ... 770 771 772]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ... 773 195 196]]
[('боорукер', 1), ('алуу', 2), ('жаман', 3), ('сүйкүмдүү', 4), ('ыраазы', 5), ('дем', 6), ('сүйүү', 7), ('ачык', 8), ('жаркыраган', 9), ('күчтүү', 10), ('ачуу', 11), ('кабыл', 12), ('сонун', 13), ('көз', 14), ('укмуштуу', 15), ('жарык', 16), ('бакыт', 17), ('кубаныч', 18), ('бактылуу', 19), ('ийгилик', 20), ('чынчылдык', 21), ('ишенимдүү', 22), ('жандуу', 23), ('сүйүктүү', 24), ('сыйкырдуу', 25), ('даяр', 26), ('кереметтүү', 27), ('татыктуу', 28), ('чын', 29), ('жүрөктөн', 30), ('катаал', 31), ('эс', 32), ('мыкты', 33), ('аң', 34), ('а

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 10, 128)           131584    
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 2)                 130       
                                                                 
Total params: 1,461,122
Trainable params: 1,461,122
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
39/39 [==============================] - 6s 37ms/step - loss: 0.6914 - accuracy: 0.5803
Epoch 2/50
39/39 [==============================] - 2s 39ms/step - loss: 0.6829

# BERT

In [2]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install keras==2.4.0

     ------------------------------------ 170.2/170.2 kB 787.9 kB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3
Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import AutoTokenizer, AutoModel
import torch
# инициализировать модель и токенизатор:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')

sent = [
    t,
    f
]
# инициализировать словарь: сохраняет токенизированные предложения
token = {'input_ids': [], 'attention_mask': []}
for sentence in sent:
    # закодировать каждое предложение, добавить в словарь
    new_token = tokenizer.encode_plus(sentence, max_length=128,
                                       truncation=True, padding='max_length',
                                       return_tensors='pt')
    token['input_ids'].append(new_token['input_ids'][0])
    token['attention_mask'].append(new_token['attention_mask'][0])
# переформатировать список тензоров в один тензор
token['input_ids'] = torch.stack(token['input_ids'])
token['attention_mask'] = torch.stack(token['attention_mask'])

print(t, f)

кутман күн, азыр баарын тактап беребиз, катачылык үчүн кечиресиз. биз катаны түшүндүк, азыр баарын тактап беребиз, кайра бул жакка чалбагыла 


In [3]:
#сохраняем токенизированные слова в словаре
token = {'input_ids': [], 'attention_mask': []}
for sentence in sent:
    #кодирование каждого предложения и добавление в словарь
    new_token = tokenizer.encode_plus(sentence, max_length=128,
                                       truncation=True, padding='max_length',
                                       return_tensors='pt')
    token['input_ids'].append(new_token['input_ids'][0])
    token['attention_mask'].append(new_token['attention_mask'][0])
#переформатировать список тензоров в один тензор
token['input_ids'] = torch.stack(token['input_ids'])
token['attention_mask'] = torch.stack(token['attention_mask'])

In [4]:
#Обработка токена через модель:
output = model(**token)
output.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [5]:
# Плотные векторные представления текста содержатся в выходном тензоре last_hidden_state
embeddings = output.last_hidden_state
embeddings

tensor([[[-0.4910,  0.7268,  1.0223,  ...,  0.2092,  0.5507,  0.7160],
         [-0.2803, -0.0674,  0.7514,  ..., -0.2142,  0.6813,  0.7216],
         [-0.0546, -0.0302,  0.7379,  ..., -0.0914,  0.3546,  0.4701],
         ...,
         [-0.1978,  0.5002,  0.7340,  ...,  0.2647,  0.2187,  0.5096],
         [-0.2144,  0.2473,  0.5777,  ...,  0.2526,  0.1348,  0.3829],
         [-0.1842,  0.2366,  0.5332,  ...,  0.2295,  0.0730,  0.2417]],

        [[-0.4984,  0.7275,  0.7603,  ...,  0.2179,  0.6096,  0.6793],
         [ 0.5962,  0.4319,  0.5483,  ...,  0.2105,  0.5040,  0.7513],
         [-0.7263,  0.2055,  0.6700,  ..., -0.0831,  0.2164,  0.2584],
         ...,
         [-0.3157,  0.3186,  0.4702,  ...,  0.2373,  0.1046,  0.2513],
         [-0.3703,  0.2820,  0.3957,  ...,  0.1724,  0.0480,  0.2230],
         [-0.2935,  0.2915,  0.4176,  ...,  0.1854,  0.0485,  0.1092]]],
       grad_fn=<NativeLayerNormBackward0>)

In [8]:
embeddings.shape

torch.Size([2, 128, 768])

In [9]:
#Изменяем размер тензора внимания:
att_mask = token['attention_mask']
att_mask.shape

torch.Size([2, 128])

In [10]:
mask = att_mask.unsqueeze(-1).expand(embeddings.size()).float()
mask.shape

torch.Size([2, 128, 768])

In [11]:
mask_embeddings = embeddings * mask
mask_embeddings.shape

torch.Size([2, 128, 768])

In [12]:
# Затем складываем оставшиеся вложения по оси 1:
summed = torch.sum(mask_embeddings, 1)
summed.shape

torch.Size([2, 768])

In [13]:
#Суммируем количество значений, которым необходимо уделить внимание в каждой позиции тензора:
summed_mask = torch.clamp(mask.sum(1), min=1e-9)
summed_mask.shape

torch.Size([2, 768])

In [14]:
mean_pooled = summed / summed_mask
mean_pooled

tensor([[-0.2863,  0.5399,  0.7446,  ...,  0.1987,  0.4188,  0.2701],
        [-0.2908,  0.6681,  0.5334,  ...,  0.1597,  0.3935,  0.1355]],
       grad_fn=<DivBackward0>)

In [15]:
from sklearn.metrics.pairwise import cosine_similarity
#Рассчитаем косинусное сходство для предложения 0:
# преобразовать из тензора PyTorch в массив numpy
mean_pooled = mean_pooled.detach().numpy()
#калькулятор
print("Первое предложение: ", t)
print("Второе предложение: ", f)
print("Эмоциональная окраска: ", res, np.argmax(res), sep='\n')
print("Эмоциональная окраска: ", res2, np.argmax(res2), sep='\n')

print("Процент сходства: ")
cosine_similarity(
    [mean_pooled[0]],
    mean_pooled[1:]
)

Первое предложение:  кутман күн, азыр баарын тактап беребиз, катачылык үчүн кечиресиз.
Второе предложение:  биз катаны түшүндүк, азыр баарын тактап беребиз, кайра бул жакка чалбагыла 
Эмоциональная окраска: 
[[0.5735018  0.42649814]]
0
Эмоциональная окраска: 
[[0.0310511 0.9689489]]
1
Процент сходства: 


array([[0.97516215]], dtype=float32)

In [16]:
import numpy as np

x = np.array([1,2,3])
a = np.array([[1,2,3],
     [1,2,3],
     [1,2,3]])
y = a @ x
print(y)


import numpy as np

x = np.array([1,2,3])
b = np.array([[[1,2,3],
     [1,2,3],
     [1,2,3]],
     [[1,2,3],
     [1,2,3],
     [1,2,3]],
     [[1,2,3],
     [1,2,3],
     [1,2,3]]])
y = x @ b @ x
print(y)

[14 14 14]
[84 84 84]


In [17]:
import numpy as np

x = np.array([1,2,3])
c = np.array([[[[1,2,3],
     [1,2,3],
     [1,2,3]],
     [[1,2,3],
     [1,2,3],
     [1,2,3]],
     [[1,2,3],
     [1,2,3],
     [1,2,3]]],
     [[[1,2,3],
     [1,2,3],
     [1,2,3]],
     [[1,2,3],
     [1,2,3],
     [1,2,3]],
     [[1,2,3],
     [1,2,3],
     [1,2,3]]],
     [[[1,2,3],
     [1,2,3],
     [1,2,3]],
     [[1,2,3],
     [1,2,3],
     [1,2,3]],
     [[1,2,3],
     [1,2,3],
     [1,2,3]]]])
y = x @ c @ x @ x
print(y)

[504 504 504]


In [18]:
#Определяет шаг нелинейно
import numpy as np

x = np.array([1,2,3])
matrix1 = np.array([[1,2,3],
     [1,2,3],
     [1,2,3]])
pace1 = np.array([[0.1,0.2,0.3],
     [0.1,0.2,0.3],
     [0.1,0.2,0.3]])

print(matrix1 + np.dot(pace1,matrix1))

[[1.6 3.2 4.8]
 [1.6 3.2 4.8]
 [1.6 3.2 4.8]]


In [19]:
import numpy as np

matrix2 = np.array([[[1,2,3],
     [1,2,3],
     [1,2,3]],
     [[1,2,3],
     [1,2,3],
     [1,2,3]],
     [[1,2,3],
     [1,2,3],
     [1,2,3]]])

pace2 = np.array([[[.1,.2,.3],
     [.1,.2,.3],
     [.1,.2,.3]],
     [[.1,.2,.3],
     [.1,.2,.3],
     [.1,.2,.3]],
     [[.1,.2,.3],
     [.1,.2,.3],
     [.1,.2,.3]]])

print(matrix2 + np.dot(pace2,matrix2))

[[[[1.6 3.2 4.8]
   [1.6 3.2 4.8]
   [1.6 3.2 4.8]]

  [[1.6 3.2 4.8]
   [1.6 3.2 4.8]
   [1.6 3.2 4.8]]

  [[1.6 3.2 4.8]
   [1.6 3.2 4.8]
   [1.6 3.2 4.8]]]


 [[[1.6 3.2 4.8]
   [1.6 3.2 4.8]
   [1.6 3.2 4.8]]

  [[1.6 3.2 4.8]
   [1.6 3.2 4.8]
   [1.6 3.2 4.8]]

  [[1.6 3.2 4.8]
   [1.6 3.2 4.8]
   [1.6 3.2 4.8]]]


 [[[1.6 3.2 4.8]
   [1.6 3.2 4.8]
   [1.6 3.2 4.8]]

  [[1.6 3.2 4.8]
   [1.6 3.2 4.8]
   [1.6 3.2 4.8]]

  [[1.6 3.2 4.8]
   [1.6 3.2 4.8]
   [1.6 3.2 4.8]]]]


In [20]:
import numpy as np

matrix3 = np.array([[[[1,2,3],
     [1,2,3],
     [1,2,3]],
     [[1,2,3],
     [1,2,3],
     [1,2,3]],
     [[1,2,3],
     [1,2,3],
     [1,2,3]]],
     [[[1,2,3],
     [1,2,3],
     [1,2,3]],
     [[1,2,3],
     [1,2,3],
     [1,2,3]],
     [[1,2,3],
     [1,2,3],
     [1,2,3]]],
     [[[1,2,3],
     [1,2,3],
     [1,2,3]],
     [[1,2,3],
     [1,2,3],
     [1,2,3]],
     [[1,2,3],
     [1,2,3],
     [1,2,3]]]])

pace3 = np.array([[[[.1,.2,.3],
     [.1,.2,.3],
     [.1,.2,.3]],
     [[.1,.2,.3],
     [.1,.2,.3],
     [.1,.2,.3]],
     [[.1,.2,.3],
     [.1,.2,.3],
     [.1,.2,.3]]],
     [[[.1,.2,.3],
     [.1,.2,.3],
     [.1,.2,.3]],
     [[.1,.2,.3],
     [.1,.2,.3],
     [.1,.2,.3]],
     [[.1,.2,.3],
     [.1,.2,.3],
     [.1,.2,.3]]],
     [[[.1,.2,.3],
     [1,2,3],
     [.1,.2,.3]],
     [[.1,.2,.3],
     [.1,.2,.3],
     [.1,.2,.3]],
     [[.1,.2,.3],
     [.1,.2,.3],
     [.1,.2,.3]]]])

print(matrix3 + np.dot(pace3,matrix3))

[[[[[[ 1.6  3.2  4.8]
     [ 1.6  3.2  4.8]
     [ 1.6  3.2  4.8]]

    [[ 1.6  3.2  4.8]
     [ 1.6  3.2  4.8]
     [ 1.6  3.2  4.8]]

    [[ 1.6  3.2  4.8]
     [ 1.6  3.2  4.8]
     [ 1.6  3.2  4.8]]]


   [[[ 1.6  3.2  4.8]
     [ 1.6  3.2  4.8]
     [ 1.6  3.2  4.8]]

    [[ 1.6  3.2  4.8]
     [ 1.6  3.2  4.8]
     [ 1.6  3.2  4.8]]

    [[ 1.6  3.2  4.8]
     [ 1.6  3.2  4.8]
     [ 1.6  3.2  4.8]]]


   [[[ 1.6  3.2  4.8]
     [ 1.6  3.2  4.8]
     [ 1.6  3.2  4.8]]

    [[ 1.6  3.2  4.8]
     [ 1.6  3.2  4.8]
     [ 1.6  3.2  4.8]]

    [[ 1.6  3.2  4.8]
     [ 1.6  3.2  4.8]
     [ 1.6  3.2  4.8]]]]



  [[[[ 1.6  3.2  4.8]
     [ 1.6  3.2  4.8]
     [ 1.6  3.2  4.8]]

    [[ 1.6  3.2  4.8]
     [ 1.6  3.2  4.8]
     [ 1.6  3.2  4.8]]

    [[ 1.6  3.2  4.8]
     [ 1.6  3.2  4.8]
     [ 1.6  3.2  4.8]]]


   [[[ 1.6  3.2  4.8]
     [ 1.6  3.2  4.8]
     [ 1.6  3.2  4.8]]

    [[ 1.6  3.2  4.8]
     [ 1.6  3.2  4.8]
     [ 1.6  3.2  4.8]]

    [[ 1.6  3.2  4.8]
     [ 1.6  3.2 

In [21]:
import numpy as np

x = np.array([1,2,3])
matrix1 = np.array([[1,2,3],
     [1,2,3],
     [1,2,3]])
pace1 = np.array([[0.1,0.2,0.3],
     [0.1,0.2,0.3],
     [0.1,0.2,0.3]])

y = (matrix1 + np.dot(pace1,matrix1)) @ x
print(y)

[22.4 22.4 22.4]


In [22]:
import numpy as np

x = np.array([1,2,3])
matrix2 = np.array([[[1,2,3],
     [1,2,3],
     [1,2,3]],
     [[1,2,3],
     [1,2,3],
     [1,2,3]],
     [[1,2,3],
     [1,2,3],
     [1,2,3]]])

pace2 = np.array([[[.1,.2,.3],
     [.1,.2,.3],
     [.1,.2,.3]],
     [[.1,.2,.3],
     [.1,.2,.3],
     [.1,.2,.3]],
     [[.1,.2,.3],
     [.1,.2,.3],
     [.1,.2,.3]]])

b = (matrix2 + np.dot(pace2,matrix2))
#y = x @ b @ x @ x
y = x @ b @ x
print(y)

[[134.4 134.4 134.4]
 [134.4 134.4 134.4]
 [134.4 134.4 134.4]]


In [23]:
import numpy as np

#b = (matrix2 + np.dot(pace2,matrix2))

x = np.array([1,2,3])
matrix2 = np.array([[[1,2,3],
     [1,2,3],
     [1,2,3]],
     [[1,2,3],
     [1,2,3],
     [1,2,3]],
     [[1,2,3],
     [1,2,3],
     [1,2,3]]])

pace2 = np.array([[[.1,.2,.3],
     [.1,.2,.3],
     [.1,.2,.3]],
     [[.1,.2,.3],
     [.1,.2,.3],
     [.1,.2,.3]],
     [[.1,.2,.3],
     [.1,.2,.3],
     [.1,.2,.3]]])

b = np.dot(pace2,matrix2)

y = b
print(y)

[[[[0.6 1.2 1.8]
   [0.6 1.2 1.8]
   [0.6 1.2 1.8]]

  [[0.6 1.2 1.8]
   [0.6 1.2 1.8]
   [0.6 1.2 1.8]]

  [[0.6 1.2 1.8]
   [0.6 1.2 1.8]
   [0.6 1.2 1.8]]]


 [[[0.6 1.2 1.8]
   [0.6 1.2 1.8]
   [0.6 1.2 1.8]]

  [[0.6 1.2 1.8]
   [0.6 1.2 1.8]
   [0.6 1.2 1.8]]

  [[0.6 1.2 1.8]
   [0.6 1.2 1.8]
   [0.6 1.2 1.8]]]


 [[[0.6 1.2 1.8]
   [0.6 1.2 1.8]
   [0.6 1.2 1.8]]

  [[0.6 1.2 1.8]
   [0.6 1.2 1.8]
   [0.6 1.2 1.8]]

  [[0.6 1.2 1.8]
   [0.6 1.2 1.8]
   [0.6 1.2 1.8]]]]
